In [1]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

pd.options.mode.chained_assignment = None  # disable the chain assignment warning

In [2]:
# Load the train and test datasets into two DataFrames
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [3]:
# view the dataset
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# Description of the dataset
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


# Check for important features
By executing the two cells below, the following conclusions can be drawn:
- Females have a higher survival rate, so we include *Sex* in our feature vector.
- Chuldren have much better survival rate than adults, so we include the *Age* feature in our feature vector.

In [5]:
# Check for survival rate based on certain features

# Passengers' survival
print('Passenger:')
print(train['Survived'].value_counts(normalize=True))

# Males' survival
print('\nMale:')
print(train['Survived'][train['Sex'] == 'male'].value_counts(normalize=True))

# Females' survival
print('\nFemale:')
print(train['Survived'][train['Sex'] == 'female'].value_counts(normalize=True))

Passenger:
0    0.616162
1    0.383838
Name: Survived, dtype: float64

Male:
0    0.811092
1    0.188908
Name: Survived, dtype: float64

Female:
1    0.742038
0    0.257962
Name: Survived, dtype: float64


In [6]:
# Check for children

# create column
train['Child'] = np.NaN

# assign values
# 1) child: age < 18
# 2) adult: age >= 18
train['Child'][train['Age'] < 18] = 1
train['Child'][train['Age'] >= 18] = 0

# Child survival
print('Children:')
print(train['Survived'][train['Child'] == 1].value_counts(normalize=True))

# Adult survival
print('\nAdult:')
print(train['Survived'][train['Child'] == 0].value_counts(normalize=True))

Children:
1    0.539823
0    0.460177
Name: Survived, dtype: float64

Adult:
0    0.618968
1    0.381032
Name: Survived, dtype: float64


## Cleaning and formatting the dataset
All the non-numeric values in the dataset have to be converted to numeric before we can do any sort of computation. All the *NaN* entries must also be filled with appropriate values.

In [7]:
# Cleaning and formatting the training data

# Convert gender to binary
# Male: 0
# Female: 1
train['Sex'][train['Sex'] == 'male'] = 0
train['Sex'][train['Sex'] == 'female'] = 1

# Fill the NaN values of Emabrked with most common value
train['Embarked'] = train['Embarked'].fillna(max(train['Embarked'].value_counts().index))

# Convert the Embarked classes to integer form
train['Embarked'][train['Embarked'] == "S"] = 0
train['Embarked'][train['Embarked'] == "C"] = 1
train['Embarked'][train['Embarked'] == "Q"] = 2

# Fill the NaN values of Age and Child with median value
train['Age'] = train['Age'].fillna(train['Age'].median())
train['Child'] = train['Child'].fillna(train['Child'].median())

In [8]:
# View the test data
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [9]:
# Describe the test set
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [10]:
# Filling up NaN entries and formatting the test data

# Impute the missing values with the median
test.Fare[152] = test['Fare'].median()
test['Age'] = test['Age'].fillna(test['Age'].median())

# Convert gender to binary
# Male: 0
# Female: 1
test['Sex'][test['Sex'] == 'male'] = 0
test['Sex'][test['Sex'] == 'female'] = 1

# Convert the Embarked classes to integer form
test['Embarked'][test['Embarked'] == "S"] = 0
test['Embarked'][test['Embarked'] == "C"] = 1
test['Embarked'][test['Embarked'] == "Q"] = 2

## Using Decision Trees
We include the following features in our feature vector
- Pclass
- Sex
- Age
- Fare

In [11]:
#  Creating a target variable
target = train['Survived'].values

# Extracting useful features from the training set
train_features = train[['Pclass', 'Sex', 'Age', 'Fare']].values

# Extract the useful features from the test set
test_features = test[['Pclass', 'Sex', 'Age', 'Fare']].values

In [12]:
# Fit the first decision tree
decision_tree = DecisionTreeClassifier()
decision_tree = decision_tree.fit(train_features, target)

In [13]:
# Importance and the score of the included features
print('Feature Importance: ', decision_tree.feature_importances_)
print('Score:', decision_tree.score(train_features, target))

Feature Importance:  [ 0.12629524  0.31274009  0.2297348   0.33122986]
Score: 0.977553310887


In [14]:
# Predict the survival rate
prediction = decision_tree.predict(test_features)

In [15]:
# Create a data frame with two columns: PassengerId & Survived.
# Survived contains the predictions
PassengerId = np.array(test['PassengerId']).astype(int)
survived = pd.DataFrame(prediction, PassengerId, columns = ['Survived'])

In [16]:
survived.head()

,Survived
892,0
893,0
894,1
895,1
896,1


In [17]:
survived.shape

(418, 1)

In [18]:
# Write the solution to a csv file
survived.to_csv("survived.csv", index_label = ["PassengerId"])

## Check for the family size
People having a large family size will have small chances of survival.  
`family_size = number of parents/children + number of siblings/spouse + the individual itself`

In [19]:
train['family_size'] = train['Parch'] + train['SibSp'] + 1
test['family_size'] = test['Parch'] + test['SibSp'] + 1
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Child,family_size
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0,0.0,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1,0.0,2
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,0.0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0,0.0,2
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0,0.0,1


In [20]:
# Create a new feature array
features_family = train[['Pclass', 'Sex', 'Age', 'Fare', 'SibSp', 'Parch', 'family_size']].values
test_features_family = test[['Pclass', 'Sex', 'Age', 'Fare', 'SibSp', 'Parch', 'family_size']].values

In [21]:
decision_tree_family = DecisionTreeClassifier()
decision_tree_family = decision_tree_family.fit(features_family, target)

In [22]:
decision_tree_family.score(features_family, target)

0.97979797979797978

In [23]:
predictions_family = decision_tree_family.predict(test_features_family)

In [24]:
# Create a data frame with two columns: PassengerId & Survived.
# Survived contains the predictions
PassengerId = np.array(test['PassengerId']).astype(int)
solution_family = pd.DataFrame(predictions_family, PassengerId, columns = ['Survived'])

In [25]:
solution_family.head()

,Survived
892,0
893,0
894,1
895,1
896,1


In [ ]:
# Write the solution to a csv file
solution_family.to_csv("solution_family.csv", index_label = ["PassengerId"])

## Using Random Forest Classifier
The Port of Embarkation also plays some important role in determining the survival of passengers. So we include the following features for the Random Forest Classifier.
- Pclass
- Age
- Sex
- Fare
- SibSp
- Parch
- Embarked

In [ ]:
features_forest = train[["Pclass", "Age", "Sex", "Fare", "SibSp", "Parch", "Embarked"]].values

In [ ]:
forest = RandomForestClassifier(
    max_depth=10, min_samples_split=2, n_estimators=100, random_state=1
)
my_forest = forest.fit(features_forest, target)

In [ ]:
# Forest score
print(my_forest.score(features_forest, target))

In [ ]:
# Compute predictions the test set features
test_features = test[["Pclass", "Age", "Sex", "Fare", "SibSp", "Parch", "Embarked"]].values
pred_forest = my_forest.predict(test_features)

In [ ]:
# print the `.feature_importances_` attribute
print(my_forest.feature_importances_)

# Compute and print the mean accuracy score
print(my_forest.score(features_forest, target))

In [ ]:
# Create a data frame with two columns: PassengerId & Survived.
# Survived contains the predictions
PassengerId = np.array(test['PassengerId']).astype(int)
solution_tree = pd.DataFrame(pred_forest, PassengerId, columns = ['Survived'])

In [ ]:
solution_tree.head()

In [ ]:
# Write the solution to a csv file
solution_tree.to_csv("solution_tree.csv", index_label = ["PassengerId"])